In [2]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *
import talib

In [3]:
key_path = key_path
project = project_id
dataset = 'bronze'
table = 'bronze_ticker_data'
table_conca = f'{project}.{dataset}.{table}'

In [4]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [5]:
data = bigquery.run_query(
        f"""
        SELECT
            * 
        FROM {table_conca} 
        """
    )

In [7]:
apple = data[data['ticker']=='AAPL']
apple

,date,open,high,low,close,adjclose,volume,ticker
0,2015-01-29 00:00:00+00:00,29.080000,29.797501,28.889999,29.725000,26.538170,337745600,AAPL
1,2015-02-25 00:00:00+00:00,32.889999,32.900002,32.037498,32.197498,28.859037,298846800,AAPL
2,2015-04-06 00:00:00+00:00,31.117500,31.877501,31.082500,31.837500,28.536367,148776000,AAPL
3,2015-04-30 00:00:00+00:00,32.160000,32.160000,31.145000,31.287500,28.043392,332781600,AAPL
4,2015-05-08 00:00:00+00:00,31.670000,31.905001,31.527500,31.905001,28.716322,222201600,AAPL
...,...,...,...,...,...,...,...,...
36681,2024-07-17 00:00:00+00:00,229.449997,231.460007,226.639999,228.880005,228.880005,57345900,AAPL
36682,2024-07-30 00:00:00+00:00,219.190002,220.330002,216.119995,218.800003,218.800003,41643800,AAPL
42922,2024-08-13 00:00:00+00:00,219.009995,221.889999,219.009995,221.270004,221.270004,44095400,AAPL
46586,2024-08-12 00:00:00+00:00,216.070007,219.509995,215.600006,217.529999,217.529999,38028100,AAPL


In [ ]:
import yfinance as yf
import pandas as pd
import talib

# Lista de tickers
tickers = ['AAPL', 'MSFT', 'GOOGL']

# Indicadores a calcular
indicators = [
    'SMA',   # Simple Moving Average
    'EMA',   # Exponential Moving Average
    'RSI',   # Relative Strength Index
    'MACD',  # Moving Average Convergence Divergence
    'ADX',   # Average Directional Index
    'BBANDS' # Bollinger Bands
]

# Función para calcular indicadores
def calculate_indicators(df):
    results = {}
    try:
        # SMA
        results['SMA'] = talib.SMA(df['Close'], timeperiod=20)
        # EMA
        results['EMA'] = talib.EMA(df['Close'], timeperiod=20)
        # RSI
        results['RSI'] = talib.RSI(df['Close'], timeperiod=14)
        # MACD
        macd, macdsignal, macdhist = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        results['MACD'] = macd
        results['MACD_Signal'] = macdsignal
        results['MACD_Hist'] = macdhist
        # ADX
        results['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)
        # BBANDS
        upperband, middleband, lowerband = talib.BBANDS(df['Close'], timeperiod=20)
        results['BBANDS_Upper'] = upperband
        results['BBANDS_Middle'] = middleband
        results['BBANDS_Lower'] = lowerband
    except Exception as e:
        print(f"Error calculating indicators: {e}")
    return results

# DataFrame para almacenar todos los resultados
all_results = []

# Obtener y procesar datos para cada ticker
for ticker in tickers:
    print(f"Processing ticker: {ticker}")
    # Descargar datos
    df = yf.download(ticker, start='2020-01-01', end='2024-01-01')
    df.reset_index(inplace=True)
    
    # Calcular indicadores
    indicators_results = calculate_indicators(df)
    
    # Añadir el ticker a los resultados
    for key, value in indicators_results.items():
        df[key] = value

    # Añadir los resultados al DataFrame final
    df['Ticker'] = ticker
    all_results.append(df)

# Concatenar todos los resultados en un solo DataFrame
final_df = pd.concat(all_results, ignore_index=True)

# Mostrar las primeras filas del DataFrame final
print(final_df.head())

# Guardar en un archivo CSV
final_df.to_csv('tickers_technical_indicators.csv', index=False)


In [11]:
# Lista de indicadores a calcular
indicators = [
    'SMA',   # Simple Moving Average
    'EMA',   # Exponential Moving Average
    'RSI',   # Relative Strength Index
    'MACD',  # Moving Average Convergence Divergence
    'ADX',   # Average Directional Index
    'BBANDS' # Bollinger Bands
]

# Diccionario para almacenar los resultados
results = {}

# Calcular los indicadores
for indicator in indicators:
    try:
        if indicator == 'SMA':
            results[indicator] = talib.SMA(apple['close'], timeperiod=20)
        elif indicator == 'EMA':
            results[indicator] = talib.EMA(apple['close'], timeperiod=20)
        elif indicator == 'RSI':
            results[indicator] = talib.RSI(apple['close'], timeperiod=14)
        elif indicator == 'MACD':
            macd, macdsignal, macdhist = talib.MACD(apple['close'], fastperiod=12, slowperiod=26, signalperiod=9)
            results['MACD'] = macd
            results['MACD_Signal'] = macdsignal
            results['MACD_Hist'] = macdhist
        elif indicator == 'ADX':
            results[indicator] = talib.ADX(apple['high'], apple['low'], apple['close'], timeperiod=14)
        elif indicator == 'BBANDS':
            upperband, middleband, lowerband = talib.BBANDS(apple['close'], timeperiod=20)
            results['BBANDS_Upper'] = upperband
            results['BBANDS_Middle'] = middleband
            results['BBANDS_Lower'] = lowerband
    except Exception as e:
        print(f"Error calculating {indicator}: {e}")

# Añadir los resultados al DataFrame
for key, value in results.items():
    apple[key] = value

apple

C:\Users\CARLES\AppData\Local\Temp\ipykernel_7788\2145920991.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apple[key] = value


,date,open,high,low,close,adjclose,volume,ticker,SMA,EMA,RSI,MACD,MACD_Signal,MACD_Hist,BBANDS_Upper,BBANDS_Middle,BBANDS_Lower,ADX
0,2015-01-29 00:00:00+00:00,29.080000,29.797501,28.889999,29.725000,26.538170,337745600,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-02-25 00:00:00+00:00,32.889999,32.900002,32.037498,32.197498,28.859037,298846800,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-04-06 00:00:00+00:00,31.117500,31.877501,31.082500,31.837500,28.536367,148776000,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-04-30 00:00:00+00:00,32.160000,32.160000,31.145000,31.287500,28.043392,332781600,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-05-08 00:00:00+00:00,31.670000,31.905001,31.527500,31.905001,28.716322,222201600,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36681,2024-07-17 00:00:00+00:00,229.449997,231.460007,226.639999,228.880005,228.880005,57345900,AAPL,189.579002,190.240675,70.717689,12.384293,9.181060,3.203232,217.468880,189.579002,161.689123,20.017367
36682,2024-07-30 00:00:00+00:00,219.190002,220.330002,216.119995,218.800003,218.800003,41643800,AAPL,191.576501,192.960611,64.574323,13.121073,9.969063,3.152010,221.736681,191.576501,161.416322,20.780514
42922,2024-08-13 00:00:00+00:00,219.009995,221.889999,219.009995,221.270004,221.270004,44095400,AAPL,193.167001,195.656744,65.368238,13.745832,10.724417,3.021416,225.953623,193.167001,160.380380,21.559946
46586,2024-08-12 00:00:00+00:00,216.070007,219.509995,215.600006,217.529999,217.529999,38028100,AAPL,195.169001,197.739911,63.063642,13.780320,11.335598,2.444723,228.761977,195.169001,161.576025,21.977319
